In [29]:
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import Point
import requests
from collections import defaultdict

In [30]:
MELB_CENTRAL = [144.96276519998628, -37.809405425615225]

In [31]:
# Your property data
property_metadata = pd.read_csv("../data/raw/properties.csv")

# Convert the coordinates column to a suitable format (assuming it's a string like '[lat, lon]')
property_metadata['coordinates'] = property_metadata['coordinates'].apply(eval)

# Create points and names from the property_metadata
coords = [Point(xy[1], xy[0]) for xy in property_metadata['coordinates']]
names = property_metadata['name'].tolist()
cost = property_metadata['weekly_rent'].tolist()
beds = property_metadata['beds'].tolist()
baths = property_metadata['baths'].tolist()
parkings = property_metadata['parkings'].tolist()

# Convert to GeoDataFrame
property = gpd.GeoDataFrame({'name': names, 'geometry': coords, 'cost': cost, 'beds': beds, 'baths': baths, 'parkings': parkings})


In [32]:
# read into shape file
lga_sf = gpd.read_file("../data/shapefile/LGA/LGA_2021_AUST_GDA2020.shp")
lga_sf['geometry'] = lga_sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
victoria_lga = lga_sf[lga_sf['STE_NAME21'] == 'Victoria']

In [33]:
# Perform a spatial join
property_LGA = gpd.sjoin(property, victoria_lga, how="inner", op="within")

/Users/henrik_hao/opt/anaconda3/envs/MAST30034/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_19586/3476730478.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs + ...

  property_LGA = gpd.sjoin(property, victoria_lga, how="inner", op="within")


In [34]:
selected_columns = ['name', 'geometry', 'cost', 'beds', 'baths', 'parkings', 'LGA_CODE21', 'LGA_NAME21']
property_LGA = property_LGA[selected_columns]
property_LGA = property_LGA.rename(columns={'LGA_CODE21': 'lga_code', 'LGA_NAME21': 'lga_name'})

In [35]:
# read into PTV station dataset
PTV_metro = gpd.read_file('../data/landing/location/PTV/PTV_metro/PTV_METRO_TRAIN_STATION.shp')
PTV_regional = gpd.read_file("../data/landing/location/PTV/PTV_regional/PTV_REGIONAL_TRAIN_STATION.shp")

In [36]:
# combine PTV stations and drop dups
PTV_station = PTV_metro.append(PTV_regional, ignore_index=True)
PTV_station = PTV_station.drop_duplicates(subset='STOP_NAME', keep='first')

/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_19586/3100524135.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  PTV_station = PTV_metro.append(PTV_regional, ignore_index=True)


In [37]:
def euclidean_distance(point1, point2):
    """
    finding out the euclidean distance between two points
    """
    return ((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2) ** 0.5

def get_distance(first_point, second_point):
    """
    get the driving distance by ORS
    """
    ORS_BASE_URL = "http://localhost:8080/ors/v2/directions/driving-car"
    url = f'{ORS_BASE_URL}?start={first_point[0]},{first_point[1]}&end={second_point[0]},{second_point[1]}'
    response = requests.get(url)
    json_response = response.json()
    
    if 'error' not in json_response:
        try:
            return json_response['features'][0]['properties']['summary']['duration'] / 60
        except KeyError: 
            # return 999 for bad response, indicating the distance is over 100km
            return 999
    return 999


def find_proximity(property_df, facility_df, name):
    """
    find out the closest facilities and add them into columns
    """
    # This will hold the name of the nearest facility for each property
    nearest_facilities = []

    # Loop through properties
    for idx1, prop_row in property_df.iterrows():
        distances = {}  # Using a dictionary to map distances to facility point
        prop_point = prop_row['geometry'].coords[0]
        
        # Loop through facilities
        for idx2, fac_row in facility_df.iterrows():
            fac_point = fac_row['geometry'].coords[0]
            # Compute the Euclidean distance
            dist = euclidean_distance(prop_point, fac_point)
            distances[dist] = fac_point

        # Finding the facility name corresponding to the shortest distance
        min_distance = min(distances.keys())
        nearest_facilities.append(get_distance(prop_point, distances[min_distance]))

    property_df['nearest_{}'.format(name)] = nearest_facilities

    return property_df

In [38]:
# find out the closest station
property_station = find_proximity(property_LGA, PTV_station, 'station')

In [39]:
property_station.head()

,name,geometry,cost,beds,baths,parkings,lga_code,lga_name,nearest_station
0,904/265 Exhibition Street Melbourne VIC 3000,POINT (144.96912 -37.80951),850.0,2.0,2.0,1.0,24600,Melbourne,1.486667
1,210/422 Collins Street Melbourne VIC 3000,POINT (144.96015 -37.81710),475.0,1.0,1.0,0.0,24600,Melbourne,2.421667
2,1902/200 Spencer Street Melbourne VIC 3000,POINT (144.95325 -37.81623),630.0,2.0,1.0,1.0,24600,Melbourne,2.486667
3,312B/399 Bourke Street Melbourne VIC 3000,POINT (144.96213 -37.81473),450.0,1.0,1.0,0.0,24600,Melbourne,2.425000
4,3313/228 La Trobe Street Melbourne VIC 3000,POINT (144.96237 -37.80961),900.0,2.0,1.0,2.0,24600,Melbourne,1.190000


In [40]:
park = gpd.read_file('../data/landing/location/Park.geojson')

In [41]:
property_station_park = find_proximity(property_station, park, 'park')

In [42]:
property_station_park.head()

,name,geometry,cost,beds,baths,parkings,lga_code,lga_name,nearest_station,nearest_park
0,904/265 Exhibition Street Melbourne VIC 3000,POINT (144.96912 -37.80951),850.0,2.0,2.0,1.0,24600,Melbourne,1.486667,4.193333
1,210/422 Collins Street Melbourne VIC 3000,POINT (144.96015 -37.81710),475.0,1.0,1.0,0.0,24600,Melbourne,2.421667,6.305000
2,1902/200 Spencer Street Melbourne VIC 3000,POINT (144.95325 -37.81623),630.0,2.0,1.0,1.0,24600,Melbourne,2.486667,7.198333
3,312B/399 Bourke Street Melbourne VIC 3000,POINT (144.96213 -37.81473),450.0,1.0,1.0,0.0,24600,Melbourne,2.425000,5.855000
4,3313/228 La Trobe Street Melbourne VIC 3000,POINT (144.96237 -37.80961),900.0,2.0,1.0,2.0,24600,Melbourne,1.190000,4.521667


In [43]:
shop = gpd.read_file('../data/landing/location/Mall, Shopping Centre & Department Store.geojson')

In [44]:
property_station_park_shop = find_proximity(property_station_park, shop, 'shop')

In [45]:
property_station_park_shop.head()

,name,geometry,cost,beds,baths,parkings,lga_code,lga_name,nearest_station,nearest_park,nearest_shop
0,904/265 Exhibition Street Melbourne VIC 3000,POINT (144.96912 -37.80951),850.0,2.0,2.0,1.0,24600,Melbourne,1.486667,4.193333,1.875000
1,210/422 Collins Street Melbourne VIC 3000,POINT (144.96015 -37.81710),475.0,1.0,1.0,0.0,24600,Melbourne,2.421667,6.305000,2.455000
2,1902/200 Spencer Street Melbourne VIC 3000,POINT (144.95325 -37.81623),630.0,2.0,1.0,1.0,24600,Melbourne,2.486667,7.198333,0.753333
3,312B/399 Bourke Street Melbourne VIC 3000,POINT (144.96213 -37.81473),450.0,1.0,1.0,0.0,24600,Melbourne,2.425000,5.855000,2.003333
4,3313/228 La Trobe Street Melbourne VIC 3000,POINT (144.96237 -37.80961),900.0,2.0,1.0,2.0,24600,Melbourne,1.190000,4.521667,2.173333


In [47]:
hospital = gpd.read_file('../data/landing/location/Hospital.geojson')

In [48]:
property_station_park_shop_hospital = find_proximity(property_station_park_shop, hospital, 'hospital')

In [49]:
property_station_park_shop_hospital.head()

,name,geometry,cost,beds,baths,parkings,lga_code,lga_name,nearest_station,nearest_park,nearest_shop,nearest_hospital
0,904/265 Exhibition Street Melbourne VIC 3000,POINT (144.96912 -37.80951),850.0,2.0,2.0,1.0,24600,Melbourne,1.486667,4.193333,1.875000,5.448333
1,210/422 Collins Street Melbourne VIC 3000,POINT (144.96015 -37.81710),475.0,1.0,1.0,0.0,24600,Melbourne,2.421667,6.305000,2.455000,5.940000
2,1902/200 Spencer Street Melbourne VIC 3000,POINT (144.95325 -37.81623),630.0,2.0,1.0,1.0,24600,Melbourne,2.486667,7.198333,0.753333,5.675000
3,312B/399 Bourke Street Melbourne VIC 3000,POINT (144.96213 -37.81473),450.0,1.0,1.0,0.0,24600,Melbourne,2.425000,5.855000,2.003333,5.488333
4,3313/228 La Trobe Street Melbourne VIC 3000,POINT (144.96237 -37.80961),900.0,2.0,1.0,2.0,24600,Melbourne,1.190000,4.521667,2.173333,4.193333


In [50]:
# convert coordinates first
school_df = pd.read_csv("../data/landing/location/complete_school_location.csv", encoding='latin-1')
school_df['coordinates'] = school_df.apply(lambda row: [row['Y'], row['X']], axis=1)

In [51]:
school_df.head()

,geometry,X,Y,School_Type,School_Name,Postal_Town,coordinates
0,POINT (145.3643874 -38.084432),145.364387,-38.084432,college,Hillcrest Christian College (Ayr Hill Campus),NaN,"[-38.084432, 145.3643874]"
1,POINT (-96.9814539 28.8155588),-96.981454,28.815559,college,Victoria College,NaN,"[28.8155588, -96.9814539]"
2,POINT (143.858787 -37.5611479),143.858787,-37.561148,college,Arts Academy,NaN,"[-37.5611479, 143.858787]"
3,POINT (144.972154 -37.8103129),144.972154,-37.810313,university,James Cook University,NaN,"[-37.8103129, 144.972154]"
4,POINT (145.1490967 -37.8768613),145.149097,-37.876861,college,Huanya Cultural Training Centre,NaN,"[-37.8768613, 145.1490967]"


In [52]:
school_metadata = school_df

# Create points and names from the school_metadata
coords = [Point(xy[1], xy[0]) for xy in school_metadata['coordinates']]
names = school_metadata['School_Type'].tolist()

# Convert to GeoDataFrame
school = gpd.GeoDataFrame({'name': names, 'geometry': coords})

In [53]:
school_SA2 = gpd.sjoin(school, victoria_lga, how="inner", op="within")

/Users/henrik_hao/opt/anaconda3/envs/MAST30034/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_19586/2203888443.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs + ...

  school_SA2 = gpd.sjoin(school, victoria_lga, how="inner", op="within")


In [54]:
school_SA2.head()

,name,geometry,index_right,LGA_CODE21,LGA_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,SHAPE_Area
0,college,POINT (145.36439 -38.08443),144,21610,Casey,2,Victoria,AUS,Australia,409.4287,http://linked.data.gov.au/dataset/asgsed3/LGA2...,1.212426,0.042056
84,Primary,POINT (145.35442 -38.04409),144,21610,Casey,2,Victoria,AUS,Australia,409.4287,http://linked.data.gov.au/dataset/asgsed3/LGA2...,1.212426,0.042056
103,Primary,POINT (145.25824 -38.00176),144,21610,Casey,2,Victoria,AUS,Australia,409.4287,http://linked.data.gov.au/dataset/asgsed3/LGA2...,1.212426,0.042056
328,Primary,POINT (145.37849 -38.20763),144,21610,Casey,2,Victoria,AUS,Australia,409.4287,http://linked.data.gov.au/dataset/asgsed3/LGA2...,1.212426,0.042056
368,Primary,POINT (145.34517 -38.00085),144,21610,Casey,2,Victoria,AUS,Australia,409.4287,http://linked.data.gov.au/dataset/asgsed3/LGA2...,1.212426,0.042056


In [55]:
property_station_park_shop_hospital_school = find_proximity(property_station_park_shop_hospital, school_SA2, 'school')

In [56]:
property_station_park_shop_hospital_school.head()

,name,geometry,cost,beds,baths,parkings,lga_code,lga_name,nearest_station,nearest_park,nearest_shop,nearest_hospital,nearest_school
0,904/265 Exhibition Street Melbourne VIC 3000,POINT (144.96912 -37.80951),850.0,2.0,2.0,1.0,24600,Melbourne,1.486667,4.193333,1.875000,5.448333,0.831667
1,210/422 Collins Street Melbourne VIC 3000,POINT (144.96015 -37.81710),475.0,1.0,1.0,0.0,24600,Melbourne,2.421667,6.305000,2.455000,5.940000,3.503333
2,1902/200 Spencer Street Melbourne VIC 3000,POINT (144.95325 -37.81623),630.0,2.0,1.0,1.0,24600,Melbourne,2.486667,7.198333,0.753333,5.675000,1.560000
3,312B/399 Bourke Street Melbourne VIC 3000,POINT (144.96213 -37.81473),450.0,1.0,1.0,0.0,24600,Melbourne,2.425000,5.855000,2.003333,5.488333,3.716667
4,3313/228 La Trobe Street Melbourne VIC 3000,POINT (144.96237 -37.80961),900.0,2.0,1.0,2.0,24600,Melbourne,1.190000,4.521667,2.173333,4.193333,1.408333


In [57]:
supermarket = gpd.read_file('../data/landing/location/Supermarket.geojson')

In [58]:
property_station_park_shop_hospital_school_supermarket = find_proximity(property_station_park_shop_hospital_school, supermarket, 'supermarket')

In [59]:
property_station_park_shop_hospital_school_supermarket.head()

,name,geometry,cost,beds,baths,parkings,lga_code,lga_name,nearest_station,nearest_park,nearest_shop,nearest_hospital,nearest_school,nearest_supermarket
0,904/265 Exhibition Street Melbourne VIC 3000,POINT (144.96912 -37.80951),850.0,2.0,2.0,1.0,24600,Melbourne,1.486667,4.193333,1.875000,5.448333,0.831667,0.743333
1,210/422 Collins Street Melbourne VIC 3000,POINT (144.96015 -37.81710),475.0,1.0,1.0,0.0,24600,Melbourne,2.421667,6.305000,2.455000,5.940000,3.503333,0.488333
2,1902/200 Spencer Street Melbourne VIC 3000,POINT (144.95325 -37.81623),630.0,2.0,1.0,1.0,24600,Melbourne,2.486667,7.198333,0.753333,5.675000,1.560000,0.813333
3,312B/399 Bourke Street Melbourne VIC 3000,POINT (144.96213 -37.81473),450.0,1.0,1.0,0.0,24600,Melbourne,2.425000,5.855000,2.003333,5.488333,3.716667,0.701667
4,3313/228 La Trobe Street Melbourne VIC 3000,POINT (144.96237 -37.80961),900.0,2.0,1.0,2.0,24600,Melbourne,1.190000,4.521667,2.173333,4.193333,1.408333,0.881667


In [60]:
to_cbd = []
for idx1, prop_row in property_LGA.iterrows():
    distances = {}  # Using a dictionary to map distances to facility point
    prop_point = prop_row['geometry'].coords[0]
        
    to_cbd.append(get_distance(prop_point, MELB_CENTRAL))

property_station_park_shop_hospital_school_supermarket['distance_to_cbd'] = to_cbd

In [61]:
property_station_park_shop_hospital_school_supermarket.to_csv("../data/curated/properties_proximity.csv")